In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [51]:
#define function that takes a list of html and returns list of text elements
def to_text_list(list):
    new_list = []
    for item in list:
        new_list.append(item.text)
    return new_list

#function that take table elements from html and returns it in a dataframe
def html_table_to_df(sheet_elements):
    sheet_headers = to_text_list(sheet_elements[0].find_all("td"))
    
    df_lift = pd.DataFrame(columns = sheet_headers)
    
    for i in range(1,len(sheet_elements)):
        sheet_data = to_text_list(sheet_elements[i].find_all("td"))
        df_lift.loc[len(df_lift)] = sheet_data
    
    return df_lift

#define function that takes input of link to ski resort, state, and country and outputs a dataframe
def df_from_resort_link(URL, resort, state, country):
    page = requests.get(URL)

    #create a beautifulsoup object with the page content and choose the appropriate parser
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find("div", class_="entry-content")

    #get the link to the google sheet
    google_sheet = results.find("a")['href']

    google_page = requests.get(google_sheet)
    google_soup = BeautifulSoup(google_page.content, "html.parser")

    google_table = google_soup.find("tbody")
    sheet_elements = google_table.find_all("tr")

    df_lifts = html_table_to_df(sheet_elements)

    #insert column for country and state to the dataframe
    df_lifts.insert(0, "Country", [country]*(len(sheet_elements)-1), True)
    df_lifts.insert(1, "State", [state]*(len(sheet_elements)-1), True)
    df_lifts.insert(2, "Resort", [resort]*(len(sheet_elements)-1), True)
    return df_lifts

In [3]:
URL = "https://liftblog.com/united-states/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

# grabs container for job postings
results = soup.find(id="post-1491")

#there are two lists on this page.  first is the states and second is sharing links
states = results.find_all("ul")[0].find_all("a")


for state in states:
    state_name = state.text.strip()
    state_link = state['href']
    print(state_name)
    print(state_link)



Alabama
https://liftblog.com/alabama/
Alaska
https://liftblog.com/alaska/
Arizona
https://liftblog.com/arizona/
California
https://liftblog.com/california/
Colorado
https://liftblog.com/colorado/
Connecticut
https://liftblog.com/connecticut/
Florida
https://liftblog.com/florida/
Georgia
https://liftblog.com/georgia/
Idaho
https://liftblog.com/idaho/
Illinois
https://liftblog.com/illinois/
Indiana
https://liftblog.com/indiana/
Iowa
https://liftblog.com/iowa/
Kansas
https://liftblog.com/kansas/
Kentucky
https://liftblog.com/kentucky/
Maine
https://liftblog.com/maine/
Maryland
https://liftblog.com/wisp-md/
Massachusetts
https://liftblog.com/massachusetts/
Michigan
https://liftblog.com/michigan/
Minnesota
https://liftblog.com/minnesota/
Mississippi
https://liftblog.com/mississippi/
Missouri
https://liftblog.com/missouri/
Montana
https://liftblog.com/montana/
Nebraska
https://liftblog.com/nebraska/
Nevada
https://liftblog.com/nevada/
New Hampshire
https://liftblog.com/new-hampshire
New Jers

In [40]:
URL = "https://liftblog.com/alaska/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

# grabs container for list of resorts
results = soup.find("div", class_="entry-content")

ski_resorts = results.find_all("ul")[0].find_all("a")

for resort in ski_resorts:
    resort_name = resort.text.strip()
    resort_link = resort['href']
    print(resort_name)
    print(resort_link)

Alyeska
https://liftblog.com/alyeska-ak/
Arctic Valley
https://liftblog.com/arctic-valley-ak/
Birch Hill
https://liftblog.com/birch-hill-ak/
Eaglecrest
https://liftblog.com/eaglecrest-ak/
Goldbelt Tram
https://liftblog.com/mt-roberts-tramway-ak/
Hillberg
https://liftblog.com/hillberg-ak/
Hilltop
https://liftblog.com/hilltop-ak/
Icy Strait Point
https://liftblog.com/icy-strait-point-ak/
Mt. Eyak
https://liftblog.com/mt-eyak-ak/
Northern Warfare Training Center
https://liftblog.com/northern-warfare-training-center-ak/
Skeetawk
https://liftblog.com/skeetawk-ak/
Skiland
https://liftblog.com/skiland-ak/


In [52]:
df_from_resort_link('https://liftblog.com/alyeska-ak/', 'Alyeska', 'Alaska', 'United States')

,Country,State,Resort,Status,Lift Name,Type,Manufacturer,Years of Operation,Capacity,Vertical Rise,Length,Horsepower,Line Speed,Chairs,Towers,Drive,Tension,Ride Time,Notes
0,United States,Alaska,Alyeska,Operating,Bear Cub,Quad,Doppelmayr CTEC,2008-,1200,288,1156,100,300,40,6,Top,Bottom,3.9,
1,United States,Alaska,Alyeska,Operating,Chair 7,Quad,Poma,1993-,2000,289,2815,100,500,94,11,Top,Top,5.6,Midstation and Loading Carpet
2,United States,Alaska,Alyeska,Operating,Glacier Bowl Express,High Speed Quad,Doppelmayr,2013-,2400,1401,4307,800,1000,92,17,Bottom,Top,4.3,
3,United States,Alaska,Alyeska,Operating,Ted's Express,High Speed Quad,Doppelmayr,2012-,1800,1348,4455,453,984,70,17,Bottom,Top,4.5,
4,United States,Alaska,Alyeska,Operating,Tram,Tram 60,VonRoll,1992-,800,2028,4699,700,2000,2,1,Bottom,Top,2.3,
5,United States,Alaska,Alyeska,Removed,1,Double,Riblet,1976-2017,970,2073,6349,,500,,,Bottom,Bottom,12.7,Midstation
6,United States,Alaska,Alyeska,Removed,1,Double,Riblet,1960-1979,,,,,,,,,,,
7,United States,Alaska,Alyeska,Removed,2,Double,Riblet,1972-1973,910,1252,3717,150,500,,,,,7.4,
8,United States,Alaska,Alyeska,Removed,3,Double,Riblet,1974-2008,1200,315,1052,,389,,,Bottom,Bottom,2.7,
9,United States,Alaska,Alyeska,Removed,4,Double,Riblet,1973-1992,1030,1120,3252,,500,,,,,6.5,


AttributeError: 'NoneType' object has no attribute 'head'

In [42]:
['state']*len(sheet_elements)

['state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state']